In [ ]:
!pip3 install gym --user
!pip3 install numpy --user
!pip3 install matplotlib --user

!pip3 install box2d --user
!pip3 install pyvirtualdisplay -U

# Q-learning

Dans le précédent workshop nous avions vue les bases du **Reinforcement learning** en resolvant L-Antique Maze via la value function. <br>
Ça été l'occasion de découvrir les diverses notions mathematiques derriere le RL (MDP, VF, etc)

Dans ce workshop, vous allez apprendre les bases de la notion du **Q-learning** et decouvrir les environnements Gym.<br>
Vous allez pour cela à la fin de ce workshop résoudre un environnement nommé [MontainCar](gym.openai.com/envs/MountainCar-v0/).

### Packages
Importons dans un premier temps les dépences suivantes:  
-numpy est le package fondamental pour le calcul scientifique avec Python.  
-matplotlib  est une librairie connue pour afficher des graphiques en Python.  
-[gym](https://pypi.org/project/gym/0.7.4/) est un outil utile lors d'usage de reinforcement learning

In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt

from validation_tests import *
from pyvirtualdisplay import Display
%matplotlib inline

# 1 - Prise en main avec l'environnement

Avant de commencer l'implémentation du Q-learning, commençons par nous familiariser avec la librairie `gym`.
Gym vous permet de tester votre agent dans un environnement. Les environnements fournis sont variés et de diverses complexités.

Celui d'aujourd'hui est nommé `MountainCar-V0`, il consiste en un vehicule situé au creux d'une colline ayant pour but de la franchir.<br>

Commençons déjà par charger et afficher notre environnement:

In [ ]:
env = gym.make('MountainCar-v0')

env.seed(1)
env.reset()

img = plt.imshow(env.render(mode='rgb_array'))
env.render()
env.close()

Très bien, nous avons affiché notre environnement.

Quelques explications:
-  `gym.make()` nous a permis de charger notre environnement
-  `env.seed()` nous permet d'avoir les mêmes résultats
-  `env.reset()` réinitialise l'environnement et retourne l'état de notre env
-  `env.render()` nous permet d'afficher notre env
-  `env.close()` ferme l'environnement

Maintenant voyons comment nous pouvons interagir avec:

In [ ]:
env = gym.make('MountainCar-v0')

env.seed(1)
env.reset()

img = plt.imshow(env.render(mode='rgb_array'))
for j in range(50):
    env.render()
    action = env.action_space.sample()
    state, reward, done, _ = env.step(action)
    if done:
        break 
env.close()

Voilà qui est plus intéressant.

Quelques explications:
-  `env.action_space.sample()` prend une action au hasard parmi celles possibles
-  `state` est votre état après l'action effectuée
-  `reward` est la récompense reçu pour avoir effectué cette action
-  `done` indique si l'environnement est terminé

Le `state` représente l'état de votre agent dans son environnement, dans le cas de l'environnement *MountainCar-v0* le `state` est composé de deux valeurs: `position` et `velocity`.<br>
Comme leur nom l'indique, `position` représente la position de l'agent dans l'environnement et `velocity` represente sa velocité à un instant $t$.

Regardons comment retrouver ses informations:

In [ ]:
print("Etat: ", env.observation_space)

print("low: ", env.observation_space.low)
print("hight: ", env.observation_space.high)

Décortiquons ces informations ensemble.

`Box()` répresente un tableau à N dimensions, il est ici de dimension $(2,)$ ce qui signifie que notre `state` est de dimsensions $(2,)$.<br>
Cela correspond bien à (`position`, `velocity`) comme nous l'avons dit précédemment.

On a aussi print les `low` et `hight` de ces deux valeurs, ce sont les deux valeurs extrêmes que peut avoir notre `state`.

`[-1.2, 0.6]` correspond à l'encradrement des valeurs de `position`.<br>
`[-0.07, 0.07]` correspond à l'encradrement des valeurs de `velocity`.

Pour en savoir plus vous pouvez voir le repo github de l'environnement [MountainCar](https://github.com/openai/gym/wiki/MountainCar-v0).

Maintenant que nous savons comment interpréter les informations de `state`, passons au contrôle de notre agent.

**Exercice:** Affichez l'espace d'action de notre agent dans son environnement.<br>
**Indice:** [doc de Gym](https://gym.openai.com/docs/)

In [ ]:
print("Action: ", )  # rajoutez votre code (~1 ligne)

**Resultat attendu:** `Action:  Discrete(3)`

Interprétons ces informations:

`Discrete()` signifie que toutes nos actions $\in N+$.<br>
Le `3` correspond au nombre d'actions effectuables par notre agent dans son environnement, dans notre cas ces actions sont: $Reculer$, $Attendre$ et $Avancer$.

# 2 - Le Q-learning


<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/678cb558a9d59c33ef4810c9618baf34a9577686">

### Nomenclature:

- $Q$ est notre `Q-table`
- $St$ représente notre `state` à un instant $t$
- $at$ représente l'action prise à un instant $t$
- $\alpha$ (alpha) est le `learning rate`, c'est l'importance que l'on donne au `new state`. <br>Sachant 0 < $\alpha$ < 1, plus le `learning rate` est élevé, plus on donne de l'importance au `new state`.
- $\gamma$ (gamma) est le `discount factor`, c'est l'importance donnée à un futur reward. <br>Sachant 0 < $\gamma$ < 1, plus le `discount factor` est élevé, plus notre agent "pensera" au long terme.
- $\epsilon$ (epsilon) est le facteur aléatoire de l'apprentissage<br>Sachant 0 < $\epsilon$ < 1, plus epsilon est élevé, plus notre agent effectuera des actions aléatoires.


### Algorithme du Q-learning

Pour commencer l'utilisateur définit sa Q-table à des valeur arbitraires.<br>

Pour rappel la Q-table contient pour chaque binome `state` $S$ et action $a$, le `reward` qui leur est associé.<br>
Si l'utilisateur se trouve dans un `state` $St$ et effectue une action $at$ il recevra en `reward` $Q[St, at]$.

Une fois la Q-table créée on commence l'algorithme:

L'utilisateur se trouve à un `state` $S$.<br>Il récupère un nombre aléatoire $r$, si $r$ < $\epsilon$ il effectue une action aléatoire, si $r$ > $\epsilon$ il effectue alors une action dite `greedy`.<br>
Apres avoir effectué l'action $a$, l'utilisateur se trouve dans un `state` $S'$ et a reçu un `reward` $r$.


L'utilisateur va pouvoir mettre à jour sa Q-table, pour rappel:
$Qnew[st, at] = Q[st, at] + \alpha * (rt  + \gamma  * maxQ(st+1, a) - Q[st, at])$

Au fur et à mesure que l'utilisateur met à jour sa Q-table, l'agent saura de mieux en mieux comment optimiser ses action pour obtenir le meilleur `reward` et ainsi résoudre son environnement.

### La pratique 

Nous avons vu la théorie, maintenant passons à la pratique.


Commençons par initialiser `states` qui contiendra tous les `states` que notre agent pourra rencontrer dans son environnement.

Pour rappel:
- `[-1.2, 0.6]` correspond à l'encradrement des valeurs de `position`.<br>
- `[-0.07, 0.07]` correspond à l'encradrement des valeurs de `velocity`.
- Notre agent obtient -1 à chaque action.
- Moins notre agent effectu d'action, plus son `reward` sera grand.

Une question se pose alors:<br>Il y a une infinité de valeurs dans l'encadrement `[-1.2, 0.6]` et l'encadrement `[-0.07, 0.07]`. Comment toutes les stocker ?<br>

Cela n'est pas possible. C'est pourquoi nous allons devoir se contenter d'un échantillons de ces valeurs.<br>
La taille de cette échantillon est variable selon la situation, dans notre cas nous allons choisir un échantillon de taille $20$.

La taille de notre échantillon est un vecteur important dans le cas du Q-Learning.
Plus l'échantillon est grand, plus notre agent sera précis mais plus il prendra du temps à tout explorer.
Il y a aussi le facteur mémoire qui rentre en jeu, la taille allouée à notre Q-table augmente de façon exponentielle:
- Un échantillon de taille 10 pour 3 actions possibles mènera à une Qtable de shape `((10, 10), 3)` et contenant $300$ valeurs.
- Un échantillon de taille 15 pour 3 actions possibles mènera à une Qtable de shape `((15, 15), 3)` et contenant $675$ valeurs.
- Un échantillon de taille 20 pour 3 actions possibles mènera à une Qtable de shape `((20, 20), 3)` et contenant $1200$ valeurs.

Ici doubler la taille de l'échantillons ne double donc pas la taille de notre Q-table mais la quadruple !

Dans notre cas voici à quoi ressemble notre échantillon:

In [ ]:
STATES_SPACE = 20

POSITION_SPACE = np.linspace(-1.2, 0.6, STATES_SPACE)
VELOCITY_SPACE = np.linspace(-0.07, 0.07, STATES_SPACE)

print(f"POSITION_SPACE: {POSITION_SPACE}\n")
print(f"VELOCITY_SPACE: {VELOCITY_SPACE}\n")

La solution la plus évidente maintenant serait de stocker dans `states[0][0]` la valeur `(-1.2)` et dans `states[0][1]`la valeur `(0.07)` etc. mais cette solution apporterait d'autres problèmes.<br>
Exemple: On se retrouve à avoir dans `states[1][0]` la valeur `-1.10526316` mais notre env ne nous retournera probablement pas la valeur exact `-1.10526316` ce qui demanderai a chaque fois d'arrondir cette dernière.

Pour contrer cela, au lieu de stocker dans `states` les valeurs de notre échantillon, nous allons stocker les index correspondant aux valeurs de notre échantillons.<br>
Exemple: `states[0]` contiendra alors `(0, 0)` correspondant respectivement à $-1,2$ et $-0.07$ dans notre échantillon.

**Exercices:** Complétez la fonction `init_states()` pour qu'elle retourne le tableau `states` de taille `STATES_SPACE` contenant toutes les combinaisons des valeurs représentants notre echantillon `(observation, velocity)`.<br>
**Indices:**
- Uttilisez `STATES_SPACE`
- `state[1]` correspondant à `(0, 1)` représente la $1ere$ valeur de notre échantillon de `observation` et la $2eme$ valeur de notre échantillon de `velocity`.
- `state[20]` correspondant à `(1, 0)` représente la $2eme$ valeur de notre échantillon de `observation` et la $1ere$ valeur de notre échantillon de `velocity`.

In [ ]:
def init_states():
    states = []
    # rajoutez votre code (~3 lignes)

    
    
    # fin de votre code
    return states

assert valide_init_states(init_states(), STATES_SPACE), "Provided function does not match requirements"

**Exercice:** Complétez la fonction `get_indexes()` pour que depuis un `state` donné en argument elle retourne les indexes correspondants dans notre échantillon.<br>
**Indices:** `np.digitize()`

In [ ]:
def get_indexes(state):
    # rajoutez votre code (~2 lignes)
    
    
    # fin de votre code
    return (position_index, velocity_index)
  
assert valide_get_indexes(get_indexes((0, 0)), POSITION_SPACE, VELOCITY_SPACE), "Provided function does not match requirements"

Passons maintenant à la gestion des actions de notre agent.

Lorsque l'on demande à notre agent d'effectuer une action qui selon lui est optimale, il effectue alors une action dite "*greedy*".<br>
C'est justement ce que vous allez implémenter.

**Exercice:** Complétez la fonction `greedy_step()` pour que depuis un `state` donné elle retourne l'action optimal à faire.

In [ ]:
ACTION_SPACE = [0, 1, 2]

def greedy_step(Q, state, actions=[0, 1, 2]):
    # rajoutez votre code (~2 lignes)
    
    
    # fin de votre code
    return action
  
assert valide_greedy_step(create_testing_Qtable()) == greedy_step(create_testing_Qtable(), (1, 2)), "Provided function does not match requirements"

Implementons maintenant l'usage d'epsilon.

**Exercice:** Complétez la fonction `take_action` pour qu'elle retourne l'action a faire à un moment $t$.<br>
**Indices:**
- Pour $r$ un nombre aléatoire $\in$ `N+`, si $r$ < $\epsilon$ alors l'action effectué sera aléatoire.
- `np.random.random()`

In [ ]:
def take_action(epsilon, Q, state, actions=[0, 1, 2]):
    # rajoutez votre code (~3 lignes)
    
    
    
    # fin de votre code

Vous pouvez maintenant passer à l'initialisation de votre Q-table.

**Exercice:** Complétez la fonction `init_Qtable() ` pour qu'elle retourne votre Q-table avec toutes ses valeurs initialisées à $0$.<br>
**Indice:**
- Votre Q-table est de shape `((STATE_SPACE, STATE_SPACE), ACTION_SPACE)`
- Qu'est ce qui est de shape `(STATE_SPACE, STATE_SPACE)` ?

In [ ]:
def init_Qtable(states, action_space):
    Q = {}
    # rajoutez votre code (~3 lignes)
    
    
    
    # fin de votre code
    return Q
  
assert valide_init_Qtable(init_states(), ACTION_SPACE) == init_Qtable(init_states(), ACTION_SPACE), "Provided function does not match requirements"

Maintenant que votre Q-table est initalisée vous devez la mettre à jour.

**Exercice:** Complétez `Qfunction` pour qu'elle retourne la nouvelle valeur de `Q[state, action` selon la *Q-function*.<br>
**Indice:** $Qnew[st, at] = Q[st, at] + \alpha * (rt  + \gamma  * maxQ(st+1, a) - Q[st, at])$

In [ ]:
def Qfunction(state, action, reward, new_state, alpha, gamma):
  # rajoutez votre code (~2 lignes)
  
  
  # fin de votre code
  return newQ

# Mise en situation

Tout est en place pour passer à la mise en situation (ou presque, nous y reviendrons).

In [ ]:
NUMBER_OF_GAMES = 2_000
ALPHA = 0.1
GAMMA = 0.99
epsilon = 1.0
EPS_MIN = 0.1
EPS_DECAY = 0.995

env = gym.make('MountainCar-v0')

env.seed(1)
np.random.seed(1)

episode_score = 0
total_rewards = np.zeros(NUMBER_OF_GAMES)
memory = []

states = init_states()
Q = init_Qtable(states, ACTION_SPACE)

**Exercice:** Complétez le code ci-dessous.

In [ ]:
for game_number in range(NUMBER_OF_GAMES):
    done = False
    obs = env.reset()
    state = get_indexes(obs)
    if game_number % 100 == 0 and game_number > 0:
        print('episode ', game_number, 'score ', episode_score, 'epsilon %.3f' % epsilon)
    episode_score = 0
    while not done:
        if (game_number % 500 == 0):
            env.render()
        # rajoutez votre code (~5 lignes)
        
        
        
        
        
        # fin de votre code
        episode_score += reward
    total_rewards[game_number] = episode_score
    epsilon = max(EPS_MIN, epsilon * EPS_DECAY)

print(f"average reward: {sum(total_rewards) / len(total_rewards)}")

In [ ]:
mean_rewards = np.zeros(NUMBER_OF_GAMES)

for t in range(NUMBER_OF_GAMES):
    mean_rewards[t] = np.mean(total_rewards[int(max(0, t - NUMBER_OF_GAMES / 10)): t + 1])

plt.plot(mean_rewards)

Super, on voit que notre agent apprend bien à résoudre son environnement mais n'y a-t-il pas un moyen d'optimiser son apprentissage ?<br>

Si ! Et vous avez déjà vu comment: en utilisant le principe d'*experience replay*.<br>
On va rajouter une mémoire à notre agent pour qu'il puisse s'entrainer plusieurs fois sur des situations qu'il pourrait rencontrer seulement rarement.

**Exercice:** Complétez la fonction `add_memory` pour qu'à chaque appel elle met à jour la mémoire avec la nouvelle valeur donnée en argument.<br>
**Indices:**
- Attention à ne pas dépasser `memory_size`
- Si on doit remplacer une valeur, on remplace la plus ancienne

In [ ]:
def add_memory(memory, state, action, reward, new_state, memory_size=600):
    # rajoutez votre code (~3 lignes)
    
    
    
    
    # fin de votre code
    return memory
  
def train_on_memory(memory, Q, alpha, gamma):
    for mem in memory:
        Q[mem['state'], mem['action']] = Qfunction(mem['state'], mem['action'], mem['reward'], mem['new_state'], alpha, gamma)
    return Q

In [ ]:
epsilon = 1.0
episode_score = 0
total_rewards = np.zeros(NUMBER_OF_GAMES)

Q = init_Qtable(states, ACTION_SPACE)
memory = []

**Exercice:** Complétez le code ci-dessous pour que votre agent utilise `memory`.

In [ ]:
for game_number in range(NUMBER_OF_GAMES):
    done = False
    obs = env.reset()
    state = get_indexes(obs)
    if game_number % 100 == 0 and game_number > 0:
        print('episode ', game_number, 'score ', episode_score, 'epsilon %.3f' % epsilon)
    episode_score = 0
    while not done:
        if (game_number % 500 == 0):
            env.render()
        # rajoutez votre code (~5 ligne)
        
        
        
        
        
        # fin de votre code
        episode_score += reward
    # rajoutez votre code (~1 ligne)
    
    # fin de votre code
    total_rewards[game_number] = episode_score
    epsilon = max(EPS_MIN, epsilon * EPS_DECAY)

print(f"average reward: {sum(total_rewards) / len(total_rewards)}")

In [ ]:
mean_rewards = np.zeros(NUMBER_OF_GAMES)

for t in range(NUMBER_OF_GAMES):
    mean_rewards[t] = np.mean(total_rewards[:])
    mean_rewards[t] = np.mean(total_rewards[int(max(0, t - NUMBER_OF_GAMES / 10)): t + 1])

plt.plot(mean_rewards)

Félicitations ! Vous avez réussi votre premier Q-Learning !

Essayez de faire de même pour [cette environnement](https://gym.openai.com/envs/CartPole-v1/) 😉